<a href="https://colab.research.google.com/github/sanaharora/sanaharora/blob/main/dataneuron_partB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install fastapi
!pip install uvicorn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [6]:
from fastapi import FastAPI
from pydantic import BaseModel

from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

import re
from tqdm import tqdm

import collections

from sklearn.cluster import KMeans

from nltk.stem import WordNetLemmatizer  # For Lemmetization of words
from nltk.corpus import stopwords  # Load list of stopwords
from nltk import word_tokenize # Convert paragraph in tokens

import pickle
import sys

from gensim.models import word2vec # For represent words in vectors
import gensim

file_path = '/content/drive/MyDrive/DataNeuron_Text_Similarity.csv'
text_data = pd.read_csv(file_path)
print("Shape of text_data : ", text_data.shape)
text_data.head(3)

text_data.isnull().sum() # Check if text data have any null values

# Preprocessing
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

!pip install nltk

import nltk
nltk.download('stopwords')

# Combining all the above stundents

preprocessed_text1 = []

# tqdm is for printing the status bar

for sentance in tqdm(text_data['text1'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text1.append(sent.lower().strip())

# Merging preprocessed_text1 in text_data

text_data['text1'] = preprocessed_text1
text_data.head(3)

# Combining all the above stundents
from tqdm import tqdm
preprocessed_text2 = []

# tqdm is for printing the status bar
for sentance in tqdm(text_data['text2'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text2.append(sent.lower().strip())

# Merging preprocessed_text2 in text_data

text_data['text2'] = preprocessed_text2

text_data.head(3)

def word_tokenizer(text):
            #tokenizes and stems the text
            tokens = word_tokenize(text)
            lemmatizer = WordNetLemmatizer()
            tokens = [lemmatizer.lemmatize(t) for t in tokens]
            return tokens

import gensim

wordmodelfile='/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz'
wordmodel= gensim.models.KeyedVectors.load_word2vec_format(wordmodelfile, binary=True)

!pip install nltk

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

!pip install --upgrade gensim

from gensim.models import KeyedVectors

similarity = [] # List for store similarity score



for ind in text_data.index:

        s1 = text_data['text1'][ind]
        s2 = text_data['text2'][ind]

        if s1==s2:
                 similarity.append(0.0) # 0 means highly similar

        else:

            s1words = word_tokenize(s1)
            s2words = word_tokenize(s2)



            vocab = wordmodel.key_to_index #the vocabulary considered in the word embeddings

            if len(s1words and s2words)==0:
                    similarity.append(1.0)

            else:

                for word in s1words.copy(): #remove sentence words not found in the vocab
                    if (word not in vocab):


                            s1words.remove(word)


                for word in s2words.copy(): #idem

                    if (word not in vocab):

                            s2words.remove(word)


                similarity.append((1-wordmodel.n_similarity(s1words, s2words)))

import pandas as pd

# Get Unique_ID and similarity

final_score = pd.DataFrame({'Similarity_score':similarity})
final_score.head(3)

final_score.to_csv('final_score.csv',index=False)

# Initialize FastAPI app
app = FastAPI()

# Define request body model
class TextPair(BaseModel):
    text1: str
    text2: str

# Define API endpoint
@app.post("/text-similarity/")
async def calculate_similarity(text_pair: TextPair):
    text1 = text_pair.text1
    text2 = text_pair.text2
    similarity_score = text_similarity(text1, text2)
    return {"similarity_score": similarity_score}


Mounted at /content/drive
Shape of text_data :  (3000, 2)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
100%|██████████| 3000/3000 [02:27<00:00, 20.41it/s]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install pyngrok

In [13]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [4]:

!pip install pyngrok
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
auth_token = "2fC653vOzhh6ih4hSW4p7Ff29Ob_7uMcy8qKZ4UJhJB3RGeAC"

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Connect to ngrok
ngrok_tunnel = ngrok.connect(8000)

# Print the public URL
print('Public URL:', ngrok_tunnel.public_url)

# Apply nest_asyncio
nest_asyncio.apply()
app= None
# Run the uvicorn server
import asyncio
async def main():
  await uvicorn.run(app,host="0.0.0.0", port=8000)
asyncio.run(main(), debug=True)

Public URL: https://bc3d-34-141-245-65.ngrok-free.app


INFO:     Started server process [8503]
INFO:     Waiting for application startup.
INFO:     ASGI 'lifespan' protocol appears unsupported.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Finished server process [8503]


KeyboardInterrupt: 

In [ ]:
import requests

# Define the URL of the API endpoint
url = "https://bc3d-34-141-245-65.ngrok-free.app/"

# Define the text pairs for similarity calculation
text_pair = {
    "text1": "First text to compare",
    "text2": "Second text to compare"
}

# Send a POST request to the API endpoint with the text pair
response = requests.post(url, json=text_pair)

# Extract the similarity score from the response
similarity_score = response.json()["similarity_score"]

# Print the similarity score
print("Similarity score:", similarity_score)
